# Prepare Environment for Biomarker Agents
In this section we prepare this notebook environment with the necessary dependencies to create all Biomarker agents.

#### Run the pip3 commands below to install all needed packages

In [ ]:
!pip3 install -r requirements.txt
!pip3 install --upgrade boto3
!pip3 show boto3

#### Import all needed Python libraries

In [ ]:
# Standard Python libraries
import boto3
import sagemaker
import os
import json
import time
import uuid
import inspect

# Import needed functions to create agent (Bedrock sample code: https://github.com/awslabs/amazon-bedrock-agent-samples/blob/main/examples/amazon-bedrock-multi-agent-collaboration/energy_efficiency_management_agent/1-energy-forecast/1_forecasting_agent.ipynb)
from bedrock_agent_helper import AgentsForAmazonBedrock

#### Extract account information needed for agent creation and define needed agent models

In [ ]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

# Define service clients
bedrock_client = boto3.client('bedrock-runtime', region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)
lambda_client = boto3.client('lambda', region)
ssm_client = boto3.client('ssm', region)

# FM used for all Biomarker Agents, this version needed for agent collaboration
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]

# Collaborator need cross-inference model to avoid throttling issues
collaborator_agent_foundation_model = ["us.anthropic.claude-3-5-sonnet-20240620-v1:0"]

account_id, region

#### Navigate to the SageMaker Notebook execution role displayed below in IAM and attach the following policies: 

#### "BedrockFullAccess", "IAMFullAccess", "AmazonSSMFullAccess", "AWSLambda_FullAccess"

Give a few minutes for these permissions to update

In [ ]:
# Output SageMaker Notebook execution role for user to add policies to

# Get the SageMaker session
sagemaker_session = sagemaker.Session()

# Get the execution role
role = sagemaker_session.get_caller_identity_arn()

print(f"SageMaker Execution Role: {role}")

#### Ensure that you have access to Bedrock models needed by the agents
If you have to enable model access, give a couple minutes before proceeding with agent creation

In [ ]:
# Create a Bedrock client
bedrock = boto3.client('bedrock')

# List the foundation models
response = bedrock.list_foundation_models()

# Save model summaries
model_summaries = response['modelSummaries']

# Extract model names
model_names = [model['modelName'] for model in model_summaries]

# Ensures that this list has the models 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' for agent use

if 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' in model_names:
    print("You have access to all models needed!")
else:
    print("Enable access to both 'Claude Sonnet 3.5' and 'Claude 3.5 Sonnet v2' models using the Bedrock console")

#### Instantiate an agents class to interact with all Bedrock Agents and have access to needed functions

In [ ]:
# Class object to interact with all agents

agents = AgentsForAmazonBedrock()
print(agents)

# Bedrock Agent Creation Flow

1. Instantiate Bedrock Agent with key characteristics (name, description, instructions, etc.)
2. Extract relevant information about agent (ARN, ID, alias, etc.)
3. Associate Knowledge Base with agent (if needed)
4. Define function/API schema for agent
5. Add needed ActionGroups with associated Lambda functions to agent
6. Give agent permission to use needed Lambda functions
7. Test sample query by invoking agent directly
8. Create official alias version ready for use by collaborator agent

# Prerequisites

This notebook assumes that you have deployed the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account. 

To properly configure the Biomarker Agents you will need the information below from your deployment:

- ncbiKnowledgeBase ID (needed for Research Evidence Agent) 

#### Fill in the NCBI Knowledge Base ID for the 'ncbi_kb_id' variable by going into Bedrock Knowledge Bases in console

In [ ]:
ncbi_kb_id = "XXXXXXXXXX"

# Create Redshift Agent
In this section we create the redshift sub-agent

#### Create Redshift Agent object

In [ ]:
# Define key parameters for agent
agent_name = 'Biomarker-database-analyst'
agent_description = "biomarker query engine with redshift"
agent_instruction = """You are a medical research assistant AI specialized in generating SQL queries for a database containing medical biomarker information. Your primary task is to interpret user queries, generate appropriate SQL queries, and provide relevant medical insights based on the data. Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 1. Before generating any SQL query, use the /getschema tool to familiarize yourself with the database structure. This will ensure your queries are correctly formatted and target the appropriate columns. 2. When generating an SQL query: a. Write the query as a single line, removing all newline ("\n") characters. b. Column names should remain consistent, do not modify the column names in the generated SQL query. 3. Before execution of a step, a. Evaluate the SQL query with the rationale of the specific step by using the /refinesql tool. Provide both the SQL query and a brief rationale for the specific step you're taking. Do not share the original user question with the tool. b. Only proceed to execute the query using the /queryredshift tool after receiving the evaluated and potentially optimized version from the /refinesql tool. c. If there is an explicit need for retrieving all the data in S3, avoid optimized query recommendations that aggregate the data. 4. When providing your response: a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to address the query. c. Ask for clarifications from the user if required."""

# Instantiate agent
redshift_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

redshift_agent

#### Extract agent information needed for later

In [ ]:
redshift_agent_id = redshift_agent[0]
redshift_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{redshift_agent_id}"

redshift_agent_id, redshift_agent_arn

#### Define the API Schema needed for the ActionGroup

In [ ]:
api_schema_string = "{\"openapi\":\"3.0.1\",\"info\":{\"title\":\"Database schema look up and query APIs\",\"version\":\"1.0.0\",\"description\":\"APIs for looking up database table schemas and making queries to database tables.\"},\"paths\":{\"/getschema\":{\"get\":{\"summary\":\"Get a list of all columns in the redshift database\",\"description\":\"Get the list of all columns in the redshift database table. Return all the column information in database table.\",\"operationId\":\"getschema\",\"responses\":{\"200\":{\"description\":\"Gets the list of table names and their schemas in the database\",\"content\":{\"application/json\":{\"schema\":{\"type\":\"array\",\"items\":{\"type\":\"object\",\"properties\":{\"Table\":{\"type\":\"string\",\"description\":\"The name of the table in the database.\"},\"Schema\":{\"type\":\"string\",\"description\":\"The schema of the table in the database. Contains all columns needed for making queries.\"}}}}}}}}}},\"/queryredshift\":{\"get\":{\"summary\":\"API to send query to the redshift database table\",\"description\":\"Send a query to the database table to retrieve information pertaining to the users question. The API takes in only one SQL query at a time, sends the SQL statement and returns the query results from the table. This API should be called for each SQL query to a database table.\",\"operationId\":\"queryredshift\",\"parameters\":[{\"name\":\"query\",\"in\":\"query\",\"required\":true,\"schema\":{\"type\":\"string\"},\"description\":\"SQL statement to query database table.\"}],\"responses\":{\"200\":{\"description\":\"Query sent successfully\",\"content\":{\"application/json\":{\"schema\":{\"type\":\"object\",\"properties\":{\"responseBody\":{\"type\":\"string\",\"description\":\"The query response from the database.\"}}}}}},\"400\":{\"description\":\"Bad request. One or more required fields are missing or invalid.\"}}}},\"/refinesql\":{\"get\":{\"summary\":\"Evaluate SQL query efficiency\",\"description\":\"Evaluate the efficiency of an SQL query based on the provided schema, query, and question.\",\"operationId\":\"refinesql\",\"parameters\":[{\"name\":\"sql\",\"in\":\"query\",\"required\":true,\"schema\":{\"type\":\"string\"},\"description\":\"The SQL query to evaluate.\"},{\"name\":\"question\",\"in\":\"query\",\"required\":true,\"schema\":{\"type\":\"string\"},\"description\":\"The question related to the rationale of the specific step.\"}],\"responses\":{\"200\":{\"description\":\"Successful response\",\"content\":{\"application/json\":{\"schema\":{\"type\":\"object\",\"properties\":{\"evaluatedQuery\":{\"type\":\"string\",\"description\":\"The evaluated SQL query, or the original query if it is efficient.\"}}}}}},\"400\":{\"description\":\"Bad request. One or more required fields are missing or invalid.\"}}}}}}"
api_schema = {"payload": api_schema_string}

#### Attach Lambda function and create sqlAction Group for agent
Note: This uses the default Lambda function name "biomarker-agent-env1", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details
redshift_lambda_function_name = "biomarker-agent-env1"  # Change if different in your account
redshift_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{redshift_lambda_function_name}"

agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=redshift_lambda_function_name,
    source_code_file=f"arn:aws:lambda:{region}:{account_id}:function:{redshift_lambda_function_name}",
    agent_action_group_name="sqlActionGroup",
    agent_action_group_description="Action for getting the database schema and querying the database",
    api_schema=api_schema,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": redshift_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": redshift_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=redshift_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=redshift_lambda_function_arn,
    StatementId="AllowRedshiftAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=redshift_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Redshift Agent Test Alias to see that it answers question properly

In [ ]:
%%time

session_id:str = str(uuid.uuid1())

query = "How many patients are current smokers?"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=query,
      agentId=redshift_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating alias so supervisor agent can interact with it

In [ ]:
redshift_agent_alias_id, redshift_agent_alias_arn = agents.create_agent_alias(
    redshift_agent[0], 'v1'
)
redshift_agent_alias_id, redshift_agent_alias_arn

# Create Research Evidence Agent
In this section we create the research evidence sub-agent

#### Create Research Evidence Agent object

In [ ]:
# Define key parameters for agent
agent_name = "Clinical-evidence-researcher"
agent_description = "Research internal and external evidence"
agent_instruction = """You are a medical research assistant AI specialized in summarizing internal and external evidence related to cancer biomarkers. Your primary task is to interpret user queries, gather internal and external evidence, and provide relevant medical insights based on the results. Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 1. When querying PubMed: a. Summarize the findings of each relevant study with citations to the specific pubmed web link of the study b. The json output will include 'Link', 'Title', 'Summary'. c. Always return the Title and Link (for example, 'https://pubmed.ncbi.nlm.nih.gov/') of each study in your response.  2. For internal evidence, make use of the knowledge base to retrieve relevant information. Always provide citations to specific content chunks. 3. When providing your response: a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. c. Separate the responses generated from internal evidence (knowledge base) and external evidence (PubMed api).  d. Conclude with a concise summary of the findings and their potential implications for medical research."""

# Instantiate agent
research_evidence_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

research_evidence_agent

#### Extract agent information needed for later

In [ ]:
research_evidence_agent_id = research_evidence_agent[0]
research_evidence_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{research_evidence_agent_id}"

research_evidence_agent_id, research_evidence_agent_arn

#### Associate Knowledge Base with agent

In [ ]:
agents.associate_kb_with_agent(
    research_evidence_agent_id,
    "Literature evidence on Relationships between Molecular and Imaging Phenotypes with Prognostic Implications", 
    ncbi_kb_id
)

#### Define the API Schema needed for the ActionGroup

In [ ]:
api_schema_string = "{\"openapi\":\"3.0.0\",\"info\":{\"title\":\"fetch biomedical literature\",\"version\":\"1.0.0\",\"description\":\"PubMed API to help answer users question using abstracts from biomedical literature.\"},\"paths\":{\"/query-pubmed\":{\"post\":{\"summary\":\"Query pubmed to relevant information from abstracts of biomedical articles.\",\"description\":\"Query pubmed to relevant information from abstracts of biomedical articles. The PubMed API takes in the user query then returns the abstracts of top 5 relevant articles.\",\"operationId\":\"query-pubmed\",\"parameters\":[{\"name\":\"query\",\"in\":\"query\",\"description\":\"user query\",\"required\":true,\"schema\":{\"type\":\"string\"}}],\"responses\":{\"200\":{\"description\":\"Query pubmed to relevant information from abstracts of biomedical articles.\",\"content\":{\"application/json\":{\"schema\":{\"type\":\"object\",\"properties\":{\"answer\":{\"type\":\"string\",\"description\":\"The response to user query with list of pubmed article abstracts.\"}}}}}}}}}}}"
api_schema = {"payload": api_schema_string}

#### Attach Lambda function and create ActionGroup for agent
Note: This uses the default Lambda function name "PubMedQueryFunction", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details
research_evidence_lambda_function_name = "PubMedQueryFunction"
research_evidence_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{research_evidence_lambda_function_name}"

agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=research_evidence_lambda_function_name,
    source_code_file=research_evidence_lambda_function_arn,
    agent_action_group_name="queryPubMed",
    agent_action_group_description="Actions for fetching biomedical literature from PubMed",
    api_schema=api_schema,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": research_evidence_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": research_evidence_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=research_evidence_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=research_evidence_lambda_function_arn,
    StatementId="AllowResearchEvidenceAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=research_evidence_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Research Evidence Agent Test Alias to see that it answers question properly

In [ ]:
session_id:str = str(uuid.uuid1())

test_query = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=research_evidence_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating alias so supervisor agent can interact with it

In [ ]:
research_evidence_agent_alias_id, research_evidence_agent_alias_arn = agents.create_agent_alias(
    research_evidence_agent[0], 'v1'
)

research_evidence_agent_alias_id, research_evidence_agent_alias_arn

# Create Medical Imaging Agent
In this section we create the medical imaging sub-agent

#### Create Medical Imaging Agent object

In [ ]:
# Define key parameters for agent
agent_name = 'Medical-imaging-expert'
agent_description = "CT scan analysis"
agent_instruction = """You are a medical research assistant AI specialized in processing medical imaging scans of patients. Your primary task is to create medical imaging jobs, or provide relevant medical insights after the jobs have completed execution. Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 1. For computed tomographic (CT) lung imaging biomarker analysis: a. Identify the patient subject ID(s) based on the conversation. b. Use the compute_imaging_biomarker tool to trigger the long-running job, passing the subject ID(s) as an array of strings (for example, ["R01-043", "R01-93"]). c. Only if specifically asked for an analysis, use the analyze_imaging_biomarker tool to process the results from the previous job. 2. When providing your response: a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. c. Present the results of the medical imaging jobs if complete.""" 

# Instantiate agent
medical_imaging_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

medical_imaging_agent

#### Extract agent information needed for later

In [ ]:
medical_imaging_agent_id = medical_imaging_agent[0]
medical_imaging_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{medical_imaging_agent_id}"

medical_imaging_agent_id, medical_imaging_agent_arn

#### Define function details for ActionGroup

In [ ]:
function_defs = [
    {
      "name": "compute_imaging_biomarker",
      "description": "compute the imaging biomarker features from lung CT scans within the tumor for a list of patient subject IDs",
      "parameters": {
        "subject_id": {
          "description": "an array of strings representing patient subject IDs, example ['R01-222', 'R01-333']",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    },
    {
      "name": "analyze_imaging_biomarker",
      "description": "analyze the result imaging biomarker features from lung CT scans within the tumor for a list of patient subject ID",
      "parameters": {
        "subject_id": {
          "description": "an array of strings representing patient subject IDs, example ['R01-222', 'R01-333']",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    }
]

#### Attach Lambda function and create ActionGroup for agent
Note: This uses the default Lambda function name "imaging-biomarker-lambda", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details, hardcode Lambda function name
medical_imaging_lambda_function_name = "imaging-biomarker-lambda"  # Change if different in your account
medical_imaging_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{medical_imaging_lambda_function_name}"

agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=medical_imaging_lambda_function_name,
    source_code_file=medical_imaging_lambda_function_arn,
    agent_action_group_name="imagingBiomarkerProcessing",
    agent_action_group_description="Actions for processing imaging biomarker within CT scans for a list of subjects",
    agent_functions=function_defs,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": medical_imaging_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": medical_imaging_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=medical_imaging_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=medical_imaging_lambda_function_arn,
    StatementId="AllowMedicalImagingAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=medical_imaging_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Medical Imaging Agent Test Alias to see that it answers question properly
Note: This agent needs a supervisor agent to properly answer some questions

In [ ]:
session_id = str(uuid.uuid1())


test_query = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=medical_imaging_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Once agent has been tested, PREPARE it to create alias so supervisor agent can interact with it

In [ ]:
medical_imaging_agent_alias_id, medical_imaging_agent_alias_arn = agents.create_agent_alias(
    medical_imaging_agent[0], 'v1'
)

medical_imaging_agent_alias_id, medical_imaging_agent_alias_arn

# Create Scientific Analysis Agent
In this section we create the scientific analysis sub-agent

#### Create Scientific Analysis Agent object

In [ ]:
# Define key parameters for agent
agent_name = 'Statistician'
agent_description = "scientific analysis for survival analysis"
agent_instruction = """You are a medical research assistant AI specialized in survival analysis with biomarkers. Your primary job is to interpret user queries, run scientific analysis tasks, and provide relevant medical insights with available visualization tools. Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 1. If the user query requires a Kaplan-Meier chart: a. Map survival status as 0 for Alive and 1 for Dead for the event parameter. b. Use survival duration as the duration parameter. c. Use the /group_survival_data tool to create baseline and condition group based on expression value threshold provided by the user. 2. If a survival regression analysis is needed: a. You need access to all records with columns start with survival status as first column, then survival duration, and the required biomarkers. b. Use the /fit_survival_regression tool to identify the best-performing biomarker based on the p-value summary. c. Ask for S3 data location if not provided, do not assume S3 bucket names or object names. 3. When you need to create a bar chart or plot: a. Always pass x_values and y_values in Array type to the function. If the user says x values are apple,egg and y values are 3,4 or as [apple,egg] and [3,4] pass their value as ['apple', 'banana'] and [3,4] 4. When providing your response: a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. c. If you generate any charts or perform statistical analyses, explain their significance in the context of the user's query. d. Conclude with a concise summary of the findings and their potential implications for medical research. e. Make sure to explain any medical or statistical concepts in a clear, accessible manner.""" 

# Instantiate agent
scientific_analysis_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

scientific_analysis_agent

#### Extract agent information needed for later

In [ ]:
scientific_analysis_agent_id = scientific_analysis_agent[0]
scientific_analysis_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{scientific_analysis_agent_id}"

scientific_analysis_agent_id, scientific_analysis_agent_arn

#### Define functions for first ActionGroup

In [ ]:
function_defs = [
    {
      "name": "bar_chart",
      "description": "create a bar chart",
      "parameters": {
        "title": {
          "description": "title of the bar chart",
          "required": True,
          "type": "string"
        },
        "x_label": {
          "description": "title of the x axis",
          "required": True,
          "type": "string"
        },
        "x_values": {
          "description": "values for the x a xis",
          "required": True,
          "type": "array"
        },
        "y_label": {
          "description": "title of the y axis",
          "required": True,
          "type": "string"
        },
        "y_values": {
          "description": "values for the y axis",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    },
]

#### Attach Lambda function and create first ActionGroup for agent
Note: This uses the default Lambda function name "MatPlotBarChartLambda", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details
scientific_analysis_lambda_function_name_1 = "MatPlotBarChartLambda"  # Change if different in your acccount
scientific_analysis_lambda_function_arn_1 = f"arn:aws:lambda:{region}:{account_id}:function:{scientific_analysis_lambda_function_name_1}"

agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=scientific_analysis_lambda_function_name_1,
    source_code_file=scientific_analysis_lambda_function_arn_1,
    agent_action_group_name="matplotbarchart",
    agent_action_group_description="Creates a bar chart from the given input values",
    agent_functions=function_defs,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": scientific_analysis_lambda_function_arn_1,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": scientific_analysis_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=scientific_analysis_lambda_function_arn_1)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=scientific_analysis_lambda_function_arn_1,
    StatementId="AllowScientificAnalysisAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=scientific_analysis_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Define functions for second ActionGroup

In [ ]:
function_defs = [
    {
      "name": "plot_kaplan_meier",
      "description": "Plots a Kaplan-Meier survival chart",
      "parameters": {
        "biomarker_name": {
          "description": "name of the biomarker",
          "required": True,
          "type": "string"
        },
        "duration_baseline": {
          "description": "duration in number of days for baseline",
          "required": True,
          "type": "array"
        },
        "duration_condition": {
          "description": "duration in number of days for condition",
          "required": True,
          "type": "array"
        },
        "event_baseline": {
          "description": "survival event for baseline",
          "required": True,
          "type": "array"
        },
        "event_condition": {
          "description": "survival event for condition",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    },
    {
      "name": "fit_survival_regression",
      "description": "Fit a survival regression model with data in a S3 object",
      "parameters": {
        "bucket": {
          "description": "s3 bucket where the data is stored by the database query tool",
          "required": True,
          "type": "string"
        },
        "key": {
          "description": "json file name that is located in the s3 bucket and contains the data for fitting the model",
          "required": True,
          "type": "string"
        }
      },
      "requireConfirmation": "DISABLED"
    }
]

#### Attach Lambda function and create second Action Group for agent
Note: This uses the default Lambda function name "ScientificPlotLambda", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details, hardcode Lambda function name
scientific_analysis_lambda_function_name_2 = "ScientificPlotLambda"  # Change if different in your account
scientific_analysis_lambda_function_arn_2 = f"arn:aws:lambda:{region}:{account_id}:function:{scientific_analysis_lambda_function_name_2}"

agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=scientific_analysis_lambda_function_name_2,
    source_code_file=scientific_analysis_lambda_function_arn_2,
    agent_action_group_name="scientificAnalysisActionGroup",
    agent_action_group_description="Actions for scientific analysis with lifelines library",
    agent_functions=function_defs,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": scientific_analysis_lambda_function_arn_2,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": scientific_analysis_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=scientific_analysis_lambda_function_arn_2)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=scientific_analysis_lambda_function_arn_2,
    StatementId="AllowScientificAnalysisAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=scientific_analysis_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Scientific Analysis Agent Test Alias to see if responds to question
Note: This agent needs a supervisor agent to properly answer some questions

In [ ]:
session_id = str(uuid.uuid1())

test_query = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=scientific_analysis_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating alias so supervisor agent can interact with it

In [ ]:
scientific_analysis_agent_alias_id, scientific_analysis_agent_alias_arn = agents.create_agent_alias(
    scientific_analysis_agent[0], 'v1'
)

scientific_analysis_agent_alias_id, scientific_analysis_agent_alias_arn

# Create Biomarker Supervisor Agent
In this section we create the biomarker supervisor agent that will interact with all the sub-agents created previously

#### Create Biomarker Supervisor Agent object

In [ ]:
# Define key parameters for agent
agent_name = "multi-agent-biomarker"
agent_description = "Multi-agent collaboration for biomarker discovery"
agent_instruction = """You are a medical research assistant AI specialized in cancer biomarker analysis and discovery. Your primary task is to interpret user queries, use relevant agents for specific tasks, and provide consolidated medical insights based on the data. Use only the appropriate agents as required by the specific question. You can provide responses from a prior agent to the next agent in sequence. To analyze patient biomarkers data, you can retrieve relevant records from the database. To find the p-value of biomarkers, a. You need to query and store all records including survival status, survival duration in years, and the required biomarkers and b. You need to fit a surival regression model with that data in S3.  When providing your response: a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. c. Present the results of individual agents d. Conclude with a concise summary of the findings and their potential implications for medical research. Make sure to explain any medical or statistical concepts in a clear, accessible manner."""

supervisor_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    collaborator_agent_foundation_model,
    agent_collaboration='SUPERVISOR',
    code_interpretation=False,
    verbose=False,
)

supervisor_agent

#### Extract agent information needed for later

In [ ]:
supervisor_agent_id = supervisor_agent[0]
supervisor_agent_id

#### Define sub-agents that supervisor is to work with created earlier in notebook

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': redshift_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in generating and executing SQL queries for a database containing medical biomarker information. Its primary task is to interpret user queries, generate and execute appropriate SQL queries. """,
        'sub_agent_association_name': 'BiomarkerDatabaseAnalyst',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': research_evidence_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in summarizing internal and external evidence related to cancer biomarkers. Its primary task is to interpret user queries, gather internal and external evidence, and provide relevant medical insights based on the results. """,
        'sub_agent_association_name': 'ClincialEvidenceResearcher',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': medical_imaging_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in processing medical imaging scans of patients. Its primary task is to create medical imaging jobs or provide relevant medical insights after jobs complete execution.  """,
        'sub_agent_association_name': 'MedicalImagingExpert',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': scientific_analysis_agent_alias_arn,
        'sub_agent_instruction': """Use this agent specialized in in survival analysis with biomarkers. Its primary job is to interpret user queries, run scientific analysis tasks, and provide relevant medical insights with available visualization tools.  """,
        'sub_agent_association_name': 'Statistician',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

In [ ]:
sub_agents_list

#### Associate sub-agents to supervisor, prepares supervisor agent

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = agents.associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn

#### Update SSM Parameter store so that Streamlit App uses new Biomarker Supervisor Agent

In [ ]:
# Update AGENT_ID for Streamlit App
ssm_client.put_parameter(
    Name='/streamlitapp/env1/AGENT_ID',
    Value=supervisor_agent_id,
    Type='String',
    Overwrite=True
)

# Update AGENT_ALIAS_ID for Streamlit App
ssm_client.put_parameter(
    Name='/streamlitapp/env1/AGENT_ALIAS_ID',
    Value=supervisor_agent_alias_id,
    Type='String',
    Overwrite=True
)

## Ask supervisor agent different question types now that sub-agents are ready! ###

In [ ]:
%%time

session_id:str = str(uuid.uuid1())


# ---------------------------- Sample Question Bank --------------------------------------------
# Redshift Agent Questions
redshift_agent_query_1 = "How many patients are current smokers?"

# Research Evidence Agent Questions
research_evidence_agent_query_1 = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk"

# Medical Imaging Agent Questions (must run in sequence)
medical_imaging_agent_query_1 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"
medical_imaging_agent_query_2 = "Can you higlight the elongation and sphericity of the tumor with these patients. Can you depict images of them?"

medical_imaging_agent_query_3 = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values and then higlight the elongation and sphericity of the tumor with these patients?"

# Scientific Analysis Agent Questions
scientific_analysis_agent_query_1 = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy, Generate a bar chart of the top 5 gene biomarkers based on their p value and include their names in the x axis.?"
# -----------------------------------------------------------------------------------------


response = bedrock_agent_runtime_client.invoke_agent(
      inputText=medical_imaging_agent_query_1, # Change value here to test different questions
      agentId=supervisor_agent_id,
      agentAliasId=supervisor_agent_alias_id, 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Supervisor Agent:\n{}".format(response))
print("====================")
print("Supervisor Agent processing query and collaborating with sub-agents to get answer")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

## Now the Biomarker Supervisor Agent is ready to assist you!

# Destroy Agents
In this section we destroy the agents created previously along any Lambda function resource policies

#### Destroy Supervisor Agent

In [ ]:
agents.delete_agent("multi-agent-biomarker",delete_role_flag=False)

#### Destroy Redshift Agent

In [ ]:
agents.delete_agent("Biomarker-database-analyst",delete_role_flag=False)

delete_request = lambda_client.remove_permission(
    FunctionName=redshift_lambda_function_arn,
    StatementId="AllowRedshiftAgentAccess"
)
print("Deleted resource-based policy statement for Lambda function {}".format(redshift_lambda_function_arn))

#### Destroy Research Evidence Agent

In [ ]:
agents.delete_agent("Clinical-evidence-researcher",delete_role_flag=False)

delete_request = lambda_client.remove_permission(
    FunctionName=research_evidence_lambda_function_arn,
    StatementId="AllowResearchEvidenceAgentAccess"
)

print("Deleted resource-based policy statement for Lambda function {}".format(research_evidence_lambda_function_arn))

#### Destroy Medical Imaging Agent

In [ ]:
agents.delete_agent("Medical-imaging-expert",delete_role_flag=False)

delete_request = lambda_client.remove_permission(
    FunctionName=medical_imaging_lambda_function_arn,
    StatementId="AllowMedicalImagingAgentAccess"
)

print("Deleted resource-based policy statement for Lambda function {}".format(medical_imaging_lambda_function_arn))

#### Destroy Scientific Analysis Agent

In [ ]:
agents.delete_agent("Statistician",delete_role_flag=False)

delete_request_1 = lambda_client.remove_permission(
    FunctionName=scientific_analysis_lambda_function_arn_1,
    StatementId="AllowScientificAnalysisAgentAccess"
)
print("Deleted resource-based policy statement for Lambda function {}".format(scientific_analysis_lambda_function_arn_1))

response = lambda_client.remove_permission(
    FunctionName=scientific_analysis_lambda_function_arn_2,
    StatementId="AllowScientificAnalysisAgentAccess"
)
print("Deleted resource-based policy statement for Lambda function {}".format(scientific_analysis_lambda_function_arn_2))